<a href="https://colab.research.google.com/github/Kevinle76/Alexa_big-data-challenge/blob/main/level1_wire1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (86.8 kB/s)
Reading package li

In [49]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-23 15:37:27--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-08-23 15:37:28 (9.64 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [52]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [69]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://mybucketkhanhle.s3.amazonaws.com/amazon_reviews_us_Wireless_v1_00.tsv"
spark.sparkContext.addFile(url)
df2 = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv"), sep='\t', header=True, inferSchema=True, timestampFormat="yyyy/MM/dd")
# df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df2.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [70]:
df2.count()

9002021

In [75]:
df2=df2.dropna()
df2.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [76]:
df2.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Transform DataFrame to fit review_id table

In [77]:
review2_df = df2.select(["review_id","customer_id","product_id","product_parent","review_date"])
review2_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [78]:
review2_df=review2_df.dropDuplicates(['review_id'])
review2_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000D29OXI03C|   13683294|B0037TV0IW|     846180035| 2014-01-31|
|R1000EGOSOZ2HM|   16191136|B00D5GC78O|     945426222| 2015-04-02|
|R1000FQ3QQ1HII|   11709408|B0071U4T40|     551702715| 2012-05-17|
|R1000GNGJ2RDWH|   43138655|B00TY1GLVI|     474167162| 2015-05-28|
|R1000TR62BELR6|    9874125|B000NVZE5M|     447732471| 2007-11-22|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [79]:
from pyspark.sql.types import * 

review2_df = review2_df.withColumn("customer_id",review2_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review2_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_df["review_date"].cast(DateType()))

review2_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



Transform DataFrame to fit products table

In [80]:
product2_df = df2.select(["product_id","product_title"])
product2_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YL0EKWE|LG G4 Case Hard T...|
|B00XK95RPQ|Selfie Stick Fibl...|
|B00SXRXUKO|Tribe AB40 Water ...|
|B009V5X1CE|RAVPower® Element...|
|B00D93OVF0|Fosmon Micro USB ...|
+----------+--------------------+
only showing top 5 rows



In [81]:
product2_df=product2_df.dropDuplicates(['product_id'])
product2_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|011040047X|Purple Hard Case ...|
|0133070565|Samsung Galaxy S ...|
|0214614700|Cool Summer Breez...|
|0214714705|Cool Summer Breez...|
|0471295418|New Trading Dimen...|
+----------+--------------------+
only showing top 5 rows



In [82]:
product2_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



Transform DataFrame to fit customers table

In [83]:
customer2_df = df2.select(["customer_id"])
customer2_df.show(5)

+-----------+
|customer_id|
+-----------+
|   16414143|
|   50800750|
|   15184378|
|   10203548|
|     488280|
+-----------+
only showing top 5 rows



In [84]:
customer2_df = customer2_df.groupBy('customer_id').count()
customer2_df.orderBy('customer_id').select('customer_id','count')
customer2_df.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   46909180|    6|
|   42560427|    7|
|   43789873|    3|
|   22037526|    2|
|   34220092|    2|
+-----------+-----+
only showing top 5 rows



In [85]:
customer2_df = customer2_df.withColumnRenamed('count', 'customer_count')
customer2_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
+-----------+--------------+
only showing top 5 rows



In [86]:
customer2_df = customer2_df.withColumn("customer_id",customer2_df["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customer2_df["customer_count"].cast(IntegerType()))

customer2_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



Transform DataFrame to fit vine table

In [87]:
vine2_df = df2.select(["review_id", "star_rating","helpful_votes", "total_votes", "vine"])
vine2_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [88]:
vine2_df=vine2_df.dropDuplicates(['review_id'])
vine2_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10000SRLJQ0VM|          5|            0|          0|   N|
|R100019BD1QTLO|          5|            0|          0|   N|
|R10004NHEEKB80|          5|            0|          0|   N|
|R10007PW29AOYE|          3|            0|          0|   N|
|R1000AZRFC5NMI|          3|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [89]:
vine2_df = vine2_df.withColumn("star_rating",vine2_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine2_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine2_df["total_votes"].cast(IntegerType()))

vine2_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Write DataFrames to Database

In [91]:
mode="append"
jdbc_url = "jdbc:postgresql://database-1.c0mjb2kp1sv1.us-east-1.rds.amazonaws.com:5432/wireless"
config = {"user":"postgres",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [93]:
review2_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [94]:
product2_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [95]:
customer2_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [96]:
vine2_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)